<a href="https://colab.research.google.com/github/krathi2k3/Bookshop-Chatbot/blob/main/Bookshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 7.84 s (started: 2024-05-15 04:44:28 +00:00)


In [ ]:
import json
import random
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

time: 78.6 ms (started: 2024-05-15 04:44:41 +00:00)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
data_file = open('sample_data/intents.json').read()
intents = json.loads(data_file)

time: 6.47 ms (started: 2024-05-15 04:44:46 +00:00)


In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

time: 957 µs (started: 2024-05-15 04:44:50 +00:00)


In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(words), "unique lemmatized words")
print (len(classes), "classes", classes)

161 documents
107 unique lemmatized words
51 classes ['Adventure stories', 'American fiction', 'Architecture', 'Art', 'Biography & Autobiography', 'Body, Mind & Spirit', 'Business & Economics', "Children's stories", 'Comics & Graphic Novels', 'Computers', 'Cooking', 'Detective and mystery stories', 'Drama', 'Education', 'English fiction', 'Family & Relationships', 'Fantasy fiction', 'Fiction', 'Foreign Language Study', 'Games', 'Health & Fitness', 'History', 'Humor', 'Juvenile Fiction', 'Juvenile Nonfiction', 'Language Arts & Disciplines', 'Law', 'Literary Collections', 'Literary Criticism', 'Medical', 'Music', 'Nature', 'Performing Arts', 'Philosophy', 'Photography', 'Poetry', 'Political Science', 'Psychology', 'Religion', 'Science', 'Science fiction', 'Self-Help', 'Social Science', 'Sports & Recreation', 'Travel', 'True Crime', 'Young Adult Fiction', 'book_search', 'goodbye', 'greeting', 'thanks']
time: 27.4 ms (started: 2024-05-15 04:44:54 +00:00)


In [ ]:
training = []
training_output_row = []
output_empty = [0] * len(classes)
for doc in documents:

    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append(bag)
    training_output_row.append(output_row)

training = np.array(training)
training_output_row = np.array(training_output_row)

from sklearn.utils import shuffle
train_x, train_y = shuffle(training, training_output_row, random_state=0)

time: 54.2 ms (started: 2024-05-15 04:44:58 +00:00)


In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

time: 168 ms (started: 2024-05-15 04:45:04 +00:00)


In [ ]:
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/500
33/33 [==============================] - 2s 2ms/step - loss: 3.9605 - accuracy: 0.0373
Epoch 2/500
33/33 [==============================] - 0s 2ms/step - loss: 3.9249 - accuracy: 0.0373
Epoch 3/500
33/33 [==============================] - 0s 2ms/step - loss: 3.8704 - accuracy: 0.0745
Epoch 4/500
33/33 [==============================] - 0s 2ms/step - loss: 3.8370 - accuracy: 0.0932
Epoch 5/500
33/33 [==============================] - 0s 2ms/step - loss: 3.7598 - accuracy: 0.0932
Epoch 6/500
33/33 [==============================] - 0s 2ms/step - loss: 3.7136 - accuracy: 0.0994
Epoch 7/500
33/33 [==============================] - 0s 2ms/step - loss: 3.6468 - accuracy: 0.1118
Epoch 8/500
33/33 [==============================] - 0s 2ms/step - loss: 3.4427 - accuracy: 0.2236
Epoch 9/500
33/33 [==============================] - 0s 3ms/step - loss: 3.3051 - accuracy: 0.1988
Epoch 10/500
33/33 [==============================] - 0s 2ms/step - loss: 3.1857 - accuracy: 0.2360
Epoch 11/

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

    return sentence_words

time: 744 µs (started: 2024-05-15 04:46:46 +00:00)


In [ ]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:

                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

time: 830 µs (started: 2024-05-15 04:47:07 +00:00)


In [ ]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    return return_list

time: 1.31 ms (started: 2024-05-15 04:47:29 +00:00)


In [ ]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break

    return result

time: 664 µs (started: 2024-05-15 04:47:41 +00:00)


In [ ]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

time: 864 µs (started: 2024-05-15 04:47:45 +00:00)


In [ ]:
chatbot_response('Recommend a book in History')

In [ ]:
chatbot_response('Hi')

In [ ]:
chatbot_response('Recommend a book in Fiction')

In [ ]:
chatbot_response('Goodbye!')